### Helpful links:

### Point Cloud Processing:

- https://github.com/szenergy/awesome-lidar
- https://github.com/PointCloudLibrary/pcl
- https://github.com/daavoo/pyntcloud
- https://github.com/isl-org/Open3D
- https://pdal.io/stages/filters.html
- https://www.danielgm.net/cc/

### Visualization:

- https://deck.gl/
- https://kepler.gl/
- https://plas.io/

## To Do:
1. Select only building footprint polygons which are relevant for the respective LiDAR tile.
2. Only save filtered .las files if they contain more than TBD points.
3. Come up with a meaningful name for .las files.

In [ ]:
import geopandas as gpd
import numpy as np
import os
import pdal
import shapely
import json
import laspy
import random

In [ ]:
%cd ..
%cd assets
assert os.getcwd() == '/Users/kevin/Projects/CS224W_LIDAR/assets', "You are not in the assets DIR"

In [ ]:
las = laspy.read('SP3278_P_11321_20171123_20171123.las')
hdr = las.header
hdr.point_count=0

# Convert LAS to numpy array (X=raw integer value x=scaled float value)
lidar_points = np.array((las.x,las.y,las.z,las.intensity, las.raw_classification,las.scan_angle_rank)).transpose()

print(f"SHAPE of LIDAR: {lidar_points.shape}")

# numpy.random.Generator.choice
rng = np.random.default_rng()

# Subsample lidar point cloud
lidar_subset = rng.choice(a=lidar_points, size=1000000, replace=False, axis=0)

print(f"SHAPE of LIDAR_SUBSET: {lidar_subset.shape}")

# Save subset to file
outfile = laspy.LasData(hdr)
outfile.x = lidar_subset[:,0]
outfile.y = lidar_subset[:,1]
outfile.z = lidar_subset[:,2]
outfile.intensity = lidar_subset[:,3]
outfile.raw_classification = lidar_subset[:,4]
outfile.scan_angle_rank = lidar_subset[:,5]

outfile.write("Subset_SP3278_P_11321_20171123_20171123.las")

In [ ]:
test_las = laspy.read('Subset_SP3278_P_11321_20171123_20171123.las')
test_points = np.array((test_las.x,test_las.y,test_las.z,test_las.intensity,test_las.raw_classification,test_las.scan_angle_rank)).transpose()
test_points.shape

In [ ]:
osm_footprints = gpd.read_file("coventry_building_footprints.geojson")
# Reproject OSM footprints to EPSG:27700
#osm_footprints = osm_footprints.to_crs("EPSG:27700")
#osm_footprints.to_file("coventry_building_footprints.geojson", driver='GeoJSON')
footprint_list = osm_footprints['geometry'].tolist()
# Filter out only Polygons and save them with their WKT string
polys = [elem.wkt for elem in footprint_list if isinstance(elem, shapely.geometry.polygon.Polygon)]
polys = random.sample(polys, 200)

In [ ]:
for idx, polygon in enumerate(polys):
    
    pipeline_definition = {

        'pipeline': [

        "Subset_SP3278_P_11321_20171123_20171123.las",

        {
            "type":"filters.crop",
            "polygon":polygon
        },

        {
            "type":"writers.las",
            "filename":f"cropped_{idx}.las"
        }

        ]
    }

    pipeline = pdal.Pipeline(json.dumps(pipeline_definition))

    pipeline.execute()

In [ ]:
for idx, polygon in enumerate(polys):

    pdal_output = laspy.read(f'cropped_{idx}.las')
    print(f'cropped_{idx}.las')
    print(pdal_output)
    print("***")